<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [21]:
## Task 1: Mark all launch sites on a map
#longitude, latitude

CCAFS_SLC_40 = [28.561857, -80.577366]
KSC_LC_39A = [28.608059, -80.603956]
VAFB_SLC_4E = [34.632093, -120.610829]

# Create a map centered around the average of the given coordinates
map_center = [34, -80.6]  # Roughly the center of the three locations
mymap = folium.Map(location=map_center, zoom_start=4)

# Add markers to the map
folium.Marker(location=CCAFS_SLC_40, popup='CCAFS SLC 40', icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker(location=KSC_LC_39A, popup='KSC LC 39A', icon=folium.Icon(color='green')).add_to(mymap)
folium.Marker(location=VAFB_SLC_4E, popup='VAFB SLC 4E', icon=folium.Icon(color='red')).add_to(mymap)

# Save the map to an HTML file
mymap


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [22]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [33]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
map_center = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]
mymap = folium.Map(location=map_center, zoom_start=4)

# Add markers and circles to the map
for index, row in launch_sites_df.iterrows():
    location = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Add a marker
    folium.Marker(
        location=location,
        popup=site_name,
        icon=folium.Icon(color='blue')
    ).add_to(mymap)
    
    # Add a circle around the marker
    folium.CircleMarker(
        location=location,
        radius=10,  # Adjust the radius as needed
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(mymap)

# Save the map to an HTML file
mymap

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [35]:
# Task 2: Mark the success/failed launches for each site on the map
# Calculate success rate for each launch site
success_rates = spacex_df.groupby('Launch Site').apply(lambda x: x['class'].mean()).reset_index(name='Success Rate')

# Merge success rates with launch site data
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()
launch_sites_df = launch_sites_df.merge(success_rates, on='Launch Site')

# Create a map centered around the average of the given coordinates
map_center = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]
mymap = folium.Map(location=map_center, zoom_start=6)

# Add markers and circles to the map
for index, row in launch_sites_df.iterrows():
    location = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    success_rate = row['Success Rate'] * 100  # Convert to percentage
    
    # Add a marker
    folium.Marker(
        location=location,
        popup=f'{site_name}<br>Success Rate: {success_rate:.2f}%',
        icon=folium.Icon(color='blue')
    ).add_to(mymap)
    
    # Add a circle around the marker
    folium.CircleMarker(
        location=location,
        radius=10,  # Adjust the radius as needed
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(mymap)

# Save the map to an HTML file
mymap


<ipython-input-35-cc8039bce2b3>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  success_rates = spacex_df.groupby('Launch Site').apply(lambda x: x['class'].mean()).reset_index(name='Success Rate')


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [36]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [37]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [39]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [40]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    location = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    
    # Create a Marker object
    marker = folium.Marker(
        location=location,
        icon=folium.Icon(color='white', icon_color=marker_color)  # Use the color from marker_color
    )
    
    # Add the Marker to the MarkerCluster
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [41]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [55]:
from math import sin, cos, sqrt, atan2, radians

coastline_points = [
    {'Lat': 28.56367, 'Lon': -80.57163},  # Near CCAFS SLC 40
    {'Lat': 28.60800, 'Lon': -80.60900},  # Near KSC LC 39A
    {'Lat': 34.63500, 'Lon': -120.62000}  # Near VAFB SLC 4E
]

# Initialize a folium map centered around the average coordinates of launch sites
map_center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=6)

# Add MarkerCluster for better visualization
marker_cluster = MarkerCluster().add_to(site_map)

# Add MousePosition to the map
MousePosition().add_to(site_map)

# List to store results
results = []

# Find the closest coastline point and calculate distances
for index, record in spacex_df.iterrows():
    launch_lat = record['Lat']
    launch_lon = record['Long']
    launch_site_name = record['Launch Site']
    
    # Initialize minimum distance to a large number
    min_distance = float('inf')
    closest_coastline = None
    
    # Find the closest coastline point
    for coast in coastline_points:
        coast_lat = coast['Lat']
        coast_lon = coast['Lon']
        distance = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast
    
    # Add markers to the map
    if closest_coastline:
        # Add marker for the launch site
        folium.Marker(
            location=[launch_lat, launch_lon],
            popup=f'{launch_site_name}\nDistance to Closest Coastline: {min_distance:.2f} km',
            icon=folium.Icon(color='blue', icon='rocket')
        ).add_to(marker_cluster)

        # Add marker for the closest coastline
        folium.Marker(
            location=[closest_coastline['Lat'], closest_coastline['Lon']],
            popup=f'Closest Coastline\nDistance: {min_distance:.2f} km',
            icon=folium.Icon(color='green', icon='info-sign')
        ).add_to(marker_cluster)

        # Add a line connecting the launch site to the closest coastline
        folium.PolyLine(
            locations=[[launch_lat, launch_lon], [closest_coastline['Lat'], closest_coastline['Lon']]],
            color='red',
            weight=2.5,
            opacity=1
        ).add_to(site_map)
    
    # Store the result
    results.append({
        'Launch Site': launch_site_name,
        'Closest Coastline Latitude': closest_coastline['Lat'],
        'Closest Coastline Longitude': closest_coastline['Lon'],
        'Distance (km)': min_distance
    })

# Create a DataFrame from the results list
closest_coastline_df = pd.DataFrame(results)

# Display the DataFrame with the closest coastline information
print(closest_coastline_df.head())

# Save the map to an HTML file
site_map


   Launch Site  Closest Coastline Latitude  Closest Coastline Longitude  \
0  CCAFS LC-40                    28.56367                    -80.57163   
1  CCAFS LC-40                    28.56367                    -80.57163   
2  CCAFS LC-40                    28.56367                    -80.57163   
3  CCAFS LC-40                    28.56367                    -80.57163   
4  CCAFS LC-40                    28.56367                    -80.57163   

   Distance (km)  
0       0.579758  
1       0.579758  
2       0.579758  
3       0.579758  
4       0.579758  


In [58]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Initialize a folium map centered around the average coordinates of launch sites
map_center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=6)

# Add MousePosition to the map
MousePosition().add_to(site_map)

# Add MarkerCluster for better visualization
marker_cluster = MarkerCluster().add_to(site_map)

# List to store results
results = []

# Find the closest coastline point and calculate distances
for index, record in spacex_df.iterrows():
    launch_lat = record['Lat']
    launch_lon = record['Long']
    launch_site_name = record['Launch Site']
    
    # Initialize minimum distance to a large number
    min_distance = float('inf')
    closest_coastline = None
    
    # Find the closest coastline point
    for coast in coastline_points:
        coast_lat = coast['Lat']
        coast_lon = coast['Lon']
        distance = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast
    
    # Add markers to the map
    if closest_coastline:
        # Add marker for the launch site
        folium.Marker(
            location=[launch_lat, launch_lon],
            popup=f'{launch_site_name}\nDistance to Closest Coastline: {min_distance:.2f} km',
            icon=folium.Icon(color='blue', icon='rocket')
        ).add_to(marker_cluster)

        # Add marker for the closest coastline with distance displayed
        distance_marker = folium.Marker(
            location=[closest_coastline['Lat'], closest_coastline['Lon']],
            icon=DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12px; color:#d35400;"><b>{min_distance:.2f} KM</b></div>',
            )
        ).add_to(marker_cluster)

        # Add a line connecting the launch site to the closest coastline
        folium.PolyLine(
            locations=[[launch_lat, launch_lon], [closest_coastline['Lat'], closest_coastline['Lon']]],
            color='red',
            weight=2.5,
            opacity=1
        ).add_to(site_map)
    
    # Store the result
    results.append({
        'Launch Site': launch_site_name,
        'Closest Coastline Latitude': closest_coastline['Lat'],
        'Closest Coastline Longitude': closest_coastline['Lon'],
        'Distance (km)': min_distance
    })

# Create a DataFrame from the results list
closest_coastline_df = pd.DataFrame(results)

# Display the DataFrame with the closest coastline information
print(closest_coastline_df.head(3))

# Save the map to an HTML file
site_map

   Launch Site  Closest Coastline Latitude  Closest Coastline Longitude  \
0  CCAFS LC-40                    28.56367                    -80.57163   
1  CCAFS LC-40                    28.56367                    -80.57163   
2  CCAFS LC-40                    28.56367                    -80.57163   

   Distance (km)  
0       0.579758  
1       0.579758  
2       0.579758  


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [60]:
points_of_interest = [
    {'Type': 'City', 'Name': 'Orlando', 'Lat': 28.538336, 'Lon': -81.379234},
    {'Type': 'Railway', 'Name': 'Orlando Amtrak', 'Lat': 28.541145, 'Lon': -81.374571},
    {'Type': 'Highway', 'Name': 'I-95', 'Lat': 28.563456, 'Lon': -80.578795},
    {'Type': 'City', 'Name': 'Santa Maria', 'Lat': 34.953033, 'Lon': -120.435734}
]

# Initialize a folium map centered around the average coordinates of launch sites
map_center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=6)

# Add MousePosition to the map
MousePosition().add_to(site_map)

# Add MarkerCluster for better visualization
marker_cluster = MarkerCluster().add_to(site_map)

# List to store results
results = []

# Find the closest point of interest and calculate distances
for index, record in spacex_df.iterrows():
    launch_lat = record['Lat']
    launch_lon = record['Long']
    launch_site_name = record['Launch Site']
    
    # Initialize minimum distance to a large number
    min_distance = float('inf')
    closest_poi = None
    
    # Find the closest point of interest
    for poi in points_of_interest:
        poi_lat = poi['Lat']
        poi_lon = poi['Lon']
        distance = calculate_distance(launch_lat, launch_lon, poi_lat, poi_lon)
        if distance < min_distance:
            min_distance = distance
            closest_poi = poi
    
    # Add markers to the map
    if closest_poi:
        # Add marker for the launch site
        folium.Marker(
            location=[launch_lat, launch_lon],
            popup=f'{launch_site_name}\nDistance to Closest {closest_poi["Type"]}: {min_distance:.2f} km',
            icon=folium.Icon(color='blue', icon='rocket')
        ).add_to(marker_cluster)

        # Add marker for the closest point of interest with distance displayed
        distance_marker = folium.Marker(
            location=[closest_poi['Lat'], closest_poi['Lon']],
            icon=DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12px; color:#d35400;"><b>{min_distance:.2f} KM to {closest_poi["Name"]}</b></div>',
            )
        ).add_to(marker_cluster)

        # Add a line connecting the launch site to the closest point of interest
        folium.PolyLine(
            locations=[[launch_lat, launch_lon], [closest_poi['Lat'], closest_poi['Lon']]],
            color='red',
            weight=2.5,
            opacity=1
        ).add_to(site_map)
    
    # Store the result
    results.append({
        'Launch Site': launch_site_name,
        'Closest POI Type': closest_poi['Type'] if closest_poi else None,
        'Closest POI Name': closest_poi['Name'] if closest_poi else None,
        'Closest POI Latitude': closest_poi['Lat'] if closest_poi else None,
        'Closest POI Longitude': closest_poi['Lon'] if closest_poi else None,
        'Distance (km)': min_distance
    })

# Create a DataFrame from the results list
closest_poi_df = pd.DataFrame(results)

# Display the DataFrame with the closest points of interest information
print(closest_poi_df.head(3))

# Save the map to an HTML file
site_map

   Launch Site Closest POI Type Closest POI Name  Closest POI Latitude  \
0  CCAFS LC-40          Highway             I-95             28.563456   
1  CCAFS LC-40          Highway             I-95             28.563456   
2  CCAFS LC-40          Highway             I-95             28.563456   

   Closest POI Longitude  Distance (km)  
0             -80.578795       0.190332  
1             -80.578795       0.190332  
2             -80.578795       0.190332  


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
